In [1]:
import os
import numpy as np
from joblib import Parallel, delayed
from myf import calculate_g_x
import pickle
from itertools import combinations
from tslearn.metrics import dtw
from tslearn.clustering import silhouette_score

In [2]:
from sktime.clustering.k_medoids import TimeSeriesKMedoids

we alread save npy, you can skip the following part (to the cell with np.save)

In [ ]:
train_files = os.listdir("smoothed_multi_train_157")
train_list = []
for train_file in train_files:
    file_path = os.path.join("smoothed_multi_train_157", train_file)
    train_list.append(np.load(file_path))
train_data = np.array(train_list)

In [53]:
train_data

array([[169.99629109, 173.17667587, 166.55681452, ..., 122.13828283,
        117.16131907, 109.759095  ],
       [100.30668828,  91.34007695,  87.84422476, ..., 108.52914288,
        105.57954702, 104.81906166],
       [135.38118147, 133.47623665, 129.13926266, ..., 151.5292107 ,
        155.72520914, 159.64878225],
       ...,
       [210.55637369, 213.97072095, 217.4587077 , ..., 225.76257176,
        215.98735536, 207.38961299],
       [221.22399297, 227.87975489, 233.78071415, ..., 199.31450247,
        192.75621684, 186.42376328],
       [ 80.72335148,  73.72034821,  68.17425233, ..., 106.53745433,
        100.66003137,  87.51722628]])

In [ ]:
# need to preprocessing data first
log_train = Parallel(n_jobs=-1)(
    delayed(calculate_g_x)(train_data[i]) for i in range(len(train_data))
)
log_train = np.array(log_train)
np.save("log_smoothed_multi_train_157.npy", log_train)

you can start from the following code

In [3]:
log_train = np.load("log_smoothed_multi_train_157.npy")

because it is time consuming to calculate every pair of dtw distances. we leverage the central limit thereom here to estimate the 20th  quantile of all the pairwise distances with sample len 157

In [4]:
def central_limit_quantile_20(
    profiles,
    n_jobs=-1,
):
    n = len(profiles)

    indices = list(combinations(range(n), 2))

    def compute_dtw(x1, x2):
        return dtw(x1, x2)

    # Calculate the score between 2 signals

    distances = Parallel(n_jobs=n_jobs)(
        delayed(compute_dtw)(profiles[i], profiles[j]) for i, j in indices
    )

    distances = np.array(distances)

    return np.nanquantile(distances, 0.2)

    # get 20th quantile of the distnaces samples

In [5]:
samples = []
for i in range(100):
    sample_indices = np.random.choice(len(log_train), size=100, replace=True)
    sampled_cases = log_train[sample_indices]
    samples.append(sampled_cases)

In [6]:
quantile_results = Parallel(n_jobs=-1)(
    delayed(central_limit_quantile_20)(sample_set) for sample_set in samples
)

In [7]:
gamma = np.mean(np.array(quantile_results))

use only use tau = 0.3*gamma here for testing, because it would run for a while

In [8]:
tau_ratios = [0.3]

In [9]:
threshold_list = []
for tau_ratio in tau_ratios:
    tau = gamma * tau_ratio
    threshold = gamma / 2 + gamma + tau  # radius upper bound
    threshold_list.append(threshold)

start training k medoid

In [10]:
log_train = np.expand_dims(log_train, axis=1)  # (n,1,m) to implement fit

In [11]:
# 2d shape (n_cases, n_timepoints)
# Example of PAM clustering
# for cluster_num, choose according to motifs num
ks = [53]
for i, k in enumerate(ks):
    km = TimeSeriesKMedoids(n_clusters=k, random_state=1)
    km.fit(log_train)
    print("end")
    # Save k_medoids to a file
    with open(f"k_medoids_multi_{threshold_list[i]}_157.pkl", "wb") as f:
        pickle.dump(km, f)

end


In [ ]:
# for validation use tukey test to remove outlier or 5-10% or smaller than certain numbers of cases
#validation_files = os.listdir("smoothed_multi_val_157")
#validation_list = []
#for validation_file in validation_files:
#    file_path = os.path.join("smoothed_multi_val_157", validation_file)
#    validation_list.append(np.load(file_path))
#validation_data = np.array(validation_list)

In [ ]:
# need to preprocessing data first
#log_val = Parallel(n_jobs=-1)(
#    delayed(calculate_g_x)(validation_data[i]) for i in range(len(validation_data))
#)
#log_val = np.array(log_val)
#np.save("log_smoothed_multi_val_157.npy", log_val)

In [12]:
log_val = np.load("log_smoothed_multi_val_157.npy")

In [13]:
log_train = np.squeeze(log_train)  # (n,1,m) to (n,m)

the following evaluation part we use only one k medoid for example

In [15]:
k_medoid_path = "k_medoids_multi_5.203641425222681_157.pkl"
threshold = float(k_medoid_path.split("_")[-2])

In [16]:
km = pickle.load(open(k_medoid_path, "rb"))

In [17]:
val_results = km.predict(log_val)

In [18]:
centers = km.cluster_centers_

In [19]:
def testing_threshold(threshold, log_val_e, val_result, centers):
    center = np.squeeze(centers[val_result])  # (1,1440) to (1440,)
    if dtw(log_val_e, center) >= threshold:
        return -1
    return val_result

In [20]:
val_results_no_outlier = Parallel(n_jobs=-1)(
    delayed(testing_threshold)(threshold, log_val[i], val_results[i], centers)
    for i in range(len(log_val))
)

In [21]:
val_results_no_outlier = np.array(val_results_no_outlier)

In [22]:
val_results_no_outlier[val_results_no_outlier == -1]

array([-1, -1, -1], dtype=int64)

In [23]:
X = np.expand_dims(log_val, axis=-1)  # n_ts, sz, d
X.shape

(4222, 157, 1)

In [24]:
silhouette_score(X, val_results, metric="dtw", n_jobs=-1)  # labels: shape = [n_ts]

0.05390101263706613